####📝 Task 1: Conversation Management with Summarization

🎯 Goal

*Implementionof a system to manage conversation history between user and assistant.
Since conversations can become very long, I will add truncation and summarization techniques to keep history concise and efficient.*

importing dependencies

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 6.5 MB/s eta 0:00:00


In [2]:
!pip install requests

In [3]:
# Imports
import json
import time
import requests
from groq import Groq


🔹 Step 1: Conversation History Storage

getting the Groq API keys

In [5]:
import os

In [6]:
#storing API in safe env variable

os.environ["GROQ_API_KEY"] = "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"

In [8]:
#intialization client

client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [28]:
# --- GLOBAL HISTORY ---
convo_history = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": "hi! how can I help you?"}
]

In [32]:
# --- CONFIG ---
MAX_TURNS = 3
SUMMARIZE_AFTER_N_TURNS = 3
MODEL_NAME = "llama-3.1-8b-instant"

In [33]:
# --- TRUNCATE FUNCTION ---
def truncate_history(history, max_turns):
    max_messages = max_turns * 2
    if len(history) > max_messages:
        history[:] = history[-max_messages:]

In [31]:

# --- CHAT LOOP ---
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chat ended.")
        break

    # 1. Add user message
    convo_history.append({"role": "user", "content": user_input})

    # 2. Truncate history (keep last N turns)
    truncate_history(convo_history, MAX_TURNS)

    # 3. Call model
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=convo_history
    )
    assistant_reply = response.choices[0].message.content

    # 4. Add assistant reply
    convo_history.append({"role": "assistant", "content": assistant_reply})

    # 5. Truncate again (after adding assistant reply)
    truncate_history(convo_history, MAX_TURNS)

    # 6. Print reply
    print(f"Assistant: {assistant_reply}")

    # 7. PERIODIC SUMMARIZATION — AFTER EVERY K-TURNS (i.e., 6 messages)
    if len(convo_history) % (SUMMARIZE_AFTER_N_TURNS * 2) == 0:
        print("\n🔄 Summarizing conversation history...\n")

        # Take ALL messages EXCEPT the very last one (which is just added assistant reply)
        # We summarize everything UP TO the last user message
        messages_to_summarize = convo_history[:-1]

        summary_prompt = {
            "role": "user",
            "content": f"""
Summarize this conversation concisely in one paragraph.
Keep only key context — names, decisions, goals, facts.
Do NOT include greetings or small talk.
Conversation:
{json.dumps(messages_to_summarize, indent=1)}
"""
        }

        try:
            summary_response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[summary_prompt]
            )
            summary_text = summary_response.choices[0].message.content.strip()

            # REPLACE ENTIRE HISTORY with summary + last assistant reply
            convo_history[:] = [
                {"role": "system", "content": f"[Summary of prior conversation: {summary_text}]"},
                convo_history[-1]  # Keep the latest assistant reply for context
            ]
            print(f"📌 Summary stored: {summary_text[:200]}...")

        except Exception as e:
            print(f"⚠️ Summary failed: {e}")

User: HIiii
Assistant: You're excited. What's on your mind? Want to chat or talk about something specific?
User: whassup budy
Assistant: Not much, just hanging out virtually. How about you? How's your day going so far?

🔄 Summarizing conversation history...

📌 Summary stored: There is no conversation to summarize as the initial exchange was composed of friendly greetings without any specific context, goals, or decisions....
User: tell me about albert camus
Assistant: Albert Camus (1913-1960) was a French philosopher, novelist, journalist, and playwright who made a profound impact on 20th-century thought. He is widely regarded as one of the most influential thinkers of his time, and his ideas continue to resonate today.

**Early Life and Influences**

Camus was born in Dréan, Algeria, to a French-Algerian family. Growing up during World War II, he was influenced by his family's modest means and the struggle for independence in Algeria, which shaped his views on social justice and the hu